In [1]:
import modal
from modal import wsgi_app

app = modal.App("tensorboard")
image = modal.Image.debian_slim(python_version="3.11").pip_install(
    "tensorboard",
    "pandas",
    "numpy",
    "Pillow",
    "matplotlib",
    "scikit-learn",
)

vol = modal.Volume.from_name("climate-forecast")

In [2]:
@app.function(
    image=image,
    volumes={"/vol": vol},
    timeout=(60 * 60 * 24),
)
@wsgi_app()
def tensorboard_app():
    import tensorboard

    board = tensorboard.program.TensorBoard()
    board.configure(logdir="/vol/flax/logs/")
    (data_provider, deprecated_multiplexer) = board._make_data_provider()
    wsgi_app = tensorboard.backend.application.TensorBoardWSGIApp(
        board.flags,
        board.plugin_loaders,
        data_provider,
        board.assets_zip_provider,
        deprecated_multiplexer,
    )  # Note: prior to April 2024, "app" was called "stub"
    return wsgi_app

In [3]:
with app.run():
  import time
  tensorboard_app.local()
  try:
      while True:
          time.sleep(1)
  except KeyboardInterrupt:
      print("Terminating app")

Output()

✓ Initialized. View run at https://modal.com/rohitp934/apps/ap-pwy9MLVi646W89pv0zhOPt

Output()

✓ Created objects.
└── 🔨 Created tensorboard_app => https://rohitp934--tensorboard-tensorboard-app-dev.modal.run

Output()

...

[MainThread] 2024-05-04T15:36:38+0000 Detected 1 background thread(s) [WSGI_0] still running after container exit. This will prevent runner shutdown for up to 30 seconds.


WARNING:modal-client:Detected 1 background thread(s) [WSGI_0] still running after container exit. This will prevent runner shutdown for up to 30 seconds.


ERROR:concurrent.futures:exception calling callback for <Future at 0x7fa6eff50e50 state=cancelled>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/concurrent/futures/_base.py", line 340, in _invoke_callbacks
    callback(self)
  File "/usr/local/lib/python3.11/asyncio/futures.py", line 401, in _call_set_state
    dest_loop.call_soon_threadsafe(_set_state, destination, source)
  File "/usr/local/lib/python3.11/asyncio/base_events.py", line 803, in call_soon_threadsafe
    self._check_closed()
  File "/usr/local/lib/python3.11/asyncio/base_events.py", line 519, in _check_closed
    raise RuntimeError('Event loop is closed')
RuntimeError: Event loop is closed


ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-94' coro=<aio_generator_output_task() done, defined at /pkg/synchronicity/async_wrap.py:27> wait_for=<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /usr/local/lib/python3.11/asyncio/futures.py:387, Task.task_wakeup()]>>


✓ App completed. View run at https://modal.com/rohitp934/apps/ap-pwy9MLVi646W89pv0zhOPt